In [1]:
import pandas as pd
import numpy as np

In [2]:
df_raw = pd.read_csv('../data/01_raw/booking.csv')

In [3]:
#sample = df.sample(frac=0.1,random_state=200)

## Data unit test for raw data

In [4]:
#!pip install great_expectations

In [5]:
import great_expectations as gx


context = gx.get_context(context_root_dir="../great_expectations")

First layer of expectations on raw data: 
- schema
- datatypes (not formatted)
- logical expectations (a booking should be for at least 1 person for at least 1 night)
- target values

In [6]:
# Create a custom expectations on sum of values in two columns
from great_expectations.expectations.expectation import ColumnPairMapExpectation


class ExpectColumnPairSumGreaterThanZero(ColumnPairMapExpectation):
    map_metric = "column_pair.sum_greater_than_zero"
    success_keys = ("column_A", "column_B")

    def validate_configuration(self, configuration):
        assert "column_A" in configuration.kwargs and "column_B" in configuration.kwargs, \
            "Must specify column_A and column_B"
        return super().validate_configuration(configuration)

    def _validate(self, configuration, metrics, runtime_configuration=None, execution_engine=None):
        df = execution_engine.get_domain_records(configuration.domain_kwargs)
        colA = configuration.kwargs["column_A"]
        colB = configuration.kwargs["column_B"]

        result_series = (df[colA] + df[colB]) > 0
        unexpected_indices = (~result_series).to_numpy().nonzero()[0].tolist()
        return {
            "success": result_series.all(),
            "result": {"unexpected_index_list": unexpected_indices}
        }

In [7]:
from great_expectations.core import ExpectationSuite, ExpectationConfiguration
import re


def build_raw_data_expectation_suite(suite_name: str = "raw_data_suite") -> ExpectationSuite:
    """
    Builds an ExpectationSuite for raw hotel booking data using schema-driven and logical validations.

    Returns:
        ExpectationSuite: A suite of expectations for the raw data.
    """
    suite = ExpectationSuite(expectation_suite_name=suite_name)

    # Column type expectations
    expected_types = {
        "Booking_ID": "object",
        "number of adults": "int64",
        "number of children": "int64",
        "number of weekend nights": "int64",
        "number of week nights": "int64",
        "type of meal": "object",
        "car parking space": "int64",
        "room type": "object",
        "lead time": "int64",
        "market segment type": "object",
        "repeated": "int64",
        "P-C": "int64",
        "P-not-C": "int64",
        "average price": "float64",
        "special requests": "int64",
        "date of reservation": "object",
        "booking status": "object"
    }

    for column, dtype in expected_types.items():
        suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_values_to_be_of_type",
                kwargs={"column": column, "type_": dtype}
            )
        )

    # Non-null expectations
    for column in expected_types.keys():
        suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_values_to_not_be_null",
                kwargs={"column": column}
            )
        )

    # No negative values in numeric columns
    non_negative_cols = [
        "number of adults", "number of children", "number of weekend nights",
        "number of week nights", "car parking space", "lead time",
        "P-C", "P-not-C", "average price", "special requests"
    ]
    for column in non_negative_cols:
        suite.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_min_to_be_between",
                kwargs={"column": column, "min_value": 0, "strict_min": False}
            )
        )
    ## Add the new custom expectations:
    suite.add_expectation(
        ExpectationConfiguration(
            expectation_type="expect_column_pair_sum_greater_than_zero",
            kwargs={"column_A": "number of adults", "column_B": "number of children"}
        )
    )
    suite.add_expectation(
        ExpectationConfiguration(
            expectation_type="expect_column_pair_sum_greater_than_zero",
            kwargs={"column_A": "number of weekend nights", "column_B": "number of week nights"}
        )
    )

    # Format of date of reservation string can be potentially parsed as date (no validity check yet)
    suite.add_expectation(
        ExpectationConfiguration(
            expectation_type="expect_column_values_to_match_regex",
            kwargs={
                "column": "date of reservation",
                "regex": r"^(\d{1,2}/\d{1,2}/\d{4}|\d{4}-\d{1,2}-\d{1,2})"
            }
        )
    )

    # Booking status should be either 'Canceled' or 'Not_Canceled'
    suite.add_expectation(
        ExpectationConfiguration(
            expectation_type="expect_column_distinct_values_to_be_in_set",
            kwargs={
                "column": "booking status",
                "value_set": ["Canceled", "Not_Canceled"]
            }
        )
    )

    # Car parking space should be 0 or 1 (to be transformed into categorical)
    suite.add_expectation(
        ExpectationConfiguration(
            expectation_type="expect_column_distinct_values_to_be_in_set",
            kwargs={
                "column": "car parking space",
                "value_set": [0, 1]
            }
        )
    )

    # Repeated should be 0 or 1 (to be transformed into categorical)
    suite.add_expectation(
        ExpectationConfiguration(
            expectation_type="expect_column_distinct_values_to_be_in_set",
            kwargs={
                "column": "repeated",
                "value_set": [0, 1]
            }
        )
    )

    return suite

In [8]:
from great_expectations.checkpoint.types.checkpoint_result import CheckpointResult
from great_expectations.core.batch import BatchRequest
from great_expectations.validator.validator import Validator
from great_expectations.dataset import PandasDataset

# Assume df_raw is your raw dataset (a pandas DataFrame)
df_raw_ge = gx.dataset.PandasDataset(df_raw)

# Build the suite
suite = build_raw_data_expectation_suite()

# Validate
results = df_raw_ge.validate(expectation_suite=suite)

# Print summary
print("Validation success:", results["success"])
for res in results["results"]:
    print(f"\nExpectation: {res['expectation_config']['expectation_type']}")
    print("✅ Success:" if res["success"] else "❌ Failed:", res["result"].get("unexpected_index_list", []))

Validation success: False

Expectation: expect_column_values_to_be_of_type
✅ Success: []

Expectation: expect_column_values_to_not_be_null
✅ Success: []

Expectation: expect_column_values_to_be_of_type
✅ Success: []

Expectation: expect_column_values_to_not_be_null
✅ Success: []

Expectation: expect_column_min_to_be_between
✅ Success: []

Expectation: expect_column_values_to_be_of_type
✅ Success: []

Expectation: expect_column_values_to_not_be_null
✅ Success: []

Expectation: expect_column_min_to_be_between
✅ Success: []

Expectation: expect_column_values_to_be_of_type
✅ Success: []

Expectation: expect_column_values_to_not_be_null
✅ Success: []

Expectation: expect_column_min_to_be_between
✅ Success: []

Expectation: expect_column_values_to_be_of_type
✅ Success: []

Expectation: expect_column_values_to_not_be_null
✅ Success: []

Expectation: expect_column_min_to_be_between
✅ Success: []

Expectation: expect_column_values_to_be_of_type
✅ Success: []

Expectation: expect_column_values_t

In [9]:
import great_expectations as gx
from great_expectations.core import ExpectationSuite

# Build your suite
suite = build_raw_data_expectation_suite()

# Wrap your DataFrame for validation
df_ge = gx.dataset.PandasDataset(df_raw)

# Validate df directly with the suite
results = df_ge.validate(expectation_suite=suite)

# Print summary with details on failures
print("Validation success:", results["success"])
for res in results["results"]:
    exp_type = res["expectation_config"]["expectation_type"]
    success = res["success"]
    unexpected = res["result"].get("unexpected_index_list", [])
    print(f"\nExpectation: {exp_type}")
    print("✅ Passed" if success else f"❌ Failed at rows: {unexpected}")

Validation success: False

Expectation: expect_column_values_to_be_of_type
✅ Passed

Expectation: expect_column_values_to_not_be_null
✅ Passed

Expectation: expect_column_values_to_be_of_type
✅ Passed

Expectation: expect_column_values_to_not_be_null
✅ Passed

Expectation: expect_column_min_to_be_between
✅ Passed

Expectation: expect_column_values_to_be_of_type
✅ Passed

Expectation: expect_column_values_to_not_be_null
✅ Passed

Expectation: expect_column_min_to_be_between
✅ Passed

Expectation: expect_column_values_to_be_of_type
✅ Passed

Expectation: expect_column_values_to_not_be_null
✅ Passed

Expectation: expect_column_min_to_be_between
✅ Passed

Expectation: expect_column_values_to_be_of_type
✅ Passed

Expectation: expect_column_values_to_not_be_null
✅ Passed

Expectation: expect_column_min_to_be_between
✅ Passed

Expectation: expect_column_values_to_be_of_type
✅ Passed

Expectation: expect_column_values_to_not_be_null
✅ Passed

Expectation: expect_column_values_to_be_of_type
✅ P

In [10]:
# debugging custom expectations
import pprint

for res in results["results"]:
    print("\n--- Full result for debugging ---")
    pprint.pprint(res["result"])


--- Full result for debugging ---
{'observed_value': 'object_'}

--- Full result for debugging ---
{'element_count': 36285,
 'partial_unexpected_list': [],
 'unexpected_count': 0,
 'unexpected_percent': 0.0,
 'unexpected_percent_total': 0.0}

--- Full result for debugging ---
{'observed_value': 'int64'}

--- Full result for debugging ---
{'element_count': 36285,
 'partial_unexpected_list': [],
 'unexpected_count': 0,
 'unexpected_percent': 0.0,
 'unexpected_percent_total': 0.0}

--- Full result for debugging ---
{'element_count': 36285,
 'missing_count': None,
 'missing_percent': None,
 'observed_value': 0}

--- Full result for debugging ---
{'observed_value': 'int64'}

--- Full result for debugging ---
{'element_count': 36285,
 'partial_unexpected_list': [],
 'unexpected_count': 0,
 'unexpected_percent': 0.0,
 'unexpected_percent_total': 0.0}

--- Full result for debugging ---
{'element_count': 36285,
 'missing_count': None,
 'missing_percent': None,
 'observed_value': 0}

--- Full r

In [11]:
print((df_raw["number of week nights"] + df_raw["number of weekend nights"]).min())

0


In [12]:
filtered_df = df_raw[(df_raw['number of week nights'] ==0 ) & (df_raw['number of weekend nights'] ==0)]
filtered_df.head()

,Booking_ID,number of adults,number of children,number of weekend nights,number of week nights,type of meal,car parking space,room type,lead time,market segment type,repeated,P-C,P-not-C,average price,special requests,date of reservation,booking status
209,INN00210,1,0,0,0,Meal Plan 1,0,Room_Type 1,4,Complementary,0,0,0,0.0,1,2/27/2018,Not_Canceled
1158,INN01159,2,0,0,0,Meal Plan 1,0,Room_Type 1,145,Online,0,0,0,0.0,1,7/5/2018,Not_Canceled
1403,INN01404,3,0,0,0,Meal Plan 1,0,Room_Type 4,57,Online,0,0,0,0.0,2,4/1/2018,Not_Canceled
1907,INN01908,2,0,0,0,Meal Plan 2,0,Room_Type 1,247,Online,0,0,0,0.0,1,6/6/2018,Not_Canceled
1986,INN01987,2,0,0,0,Meal Plan 1,0,Room_Type 1,43,Online,0,0,0,0.0,1,10/17/2017,Not_Canceled


In [13]:
filtered_df.shape

(78, 17)

78 bookings are for 0 week nights and 0 weekend nights - invalid bookings

## Data Split

In [14]:
def split_data(df, cutoff_date):
    df = df.copy()

    ref_data = df[df['date of reservation'] <= cutoff_date]
    ana_data = df[df['date of reservation'] > cutoff_date]

    return ref_data, ana_data

In [15]:
ref_data, ana_data = split_data(df_raw, "9/19/2018")

In [16]:
ref_data.shape

(33431, 17)

In [17]:
ana_data.shape

(2854, 17)

## Preprocessing Train

In [18]:
from typing import Any, Dict, Tuple
import pandas as pd

def clean_data(
    data: pd.DataFrame, params: Dict
) -> Tuple[pd.DataFrame, Dict, Dict]:    

    df_transformed = data.copy()

    # drop raws NaN values (including NaT with invalid dates if any remained)
    df_transformed = df_transformed.dropna()


    # cast repeated and car parking space as boolean
    df_transformed["car_parking_space"] = df_transformed["car_parking_space"].astype(bool)
    df_transformed["repeated"] = df_transformed["repeated"].astype(bool)

    # drop invalid bookings (with 0 sum of week and weekend nights, 0 sum of numner of children and adults)
    df_transformed = df_transformed[~((df_transformed['number_of_week_nights'] == 0) & (df_transformed['number_of_weekend_nights'] == 0))]
    df_transformed = df_transformed[~((df_transformed['number_of_children'] == 0) & (df_transformed['number_of_adults'] == 0))]

    # remove outliers
    for cols in ["lead_time", "average_price"]:
        Q1 = df_transformed[cols].quantile(0.25)
        Q3 = df_transformed[cols].quantile(0.75)
        IQR = Q3 - Q1     

        filter = (df_transformed[cols] >= Q1 - 1.5 * IQR) & (df_transformed[cols] <= Q3 + 1.5 *IQR)
        df_transformed = df_transformed.loc[filter]

    # drop columns deemed uninformative in EDA
    cols_to_drop = params["preprocessing"]["drop_columns"]
    df_transformed.drop(columns=cols_to_drop, inplace=True, errors="ignore")

    # temporarily convert date_of_reservation to string to avoid JSON serialization error
    df_temp = df_transformed.copy()
    df_temp['date_of_reservation'] = df_temp['date_of_reservation'].astype(str)
    describe_to_dict_verified = df_temp.describe(include='all').to_dict()

    return df_transformed, describe_to_dict_verified

In [19]:
params = {'preprocessing': {'drop_columns': ["car_parking_space", "repeated", "p-c", "p-not-c"]}}

In [20]:
ref_data.columns = ref_data.columns.str.replace(' ', '_').str.lower()
ref_data['date_of_reservation'] = pd.to_datetime(ref_data['date_of_reservation'], errors='coerce')

# drop raws NaN values (including NaT with invalid dates)
ref_data = ref_data.dropna(subset=["date_of_reservation"])
ref_data.head()

,booking_id,number_of_adults,number_of_children,number_of_weekend_nights,number_of_week_nights,type_of_meal,car_parking_space,room_type,lead_time,market_segment_type,repeated,p-c,p-not-c,average_price,special_requests,date_of_reservation,booking_status
0,INN00001,1,1,2,5,Meal Plan 1,0,Room_Type 1,224,Offline,0,0,0,88.00,0,2015-10-02,Not_Canceled
1,INN00002,1,0,1,3,Not Selected,0,Room_Type 1,5,Online,0,0,0,106.68,1,2018-11-06,Not_Canceled
2,INN00003,2,1,1,3,Meal Plan 1,0,Room_Type 1,1,Online,0,0,0,50.00,0,2018-02-28,Canceled
3,INN00004,1,0,0,2,Meal Plan 1,0,Room_Type 1,211,Online,0,0,0,100.00,1,2017-05-20,Canceled
4,INN00005,1,0,1,2,Not Selected,0,Room_Type 1,48,Online,0,0,0,77.00,0,2018-04-11,Canceled


In [21]:
df_clean, describe_dict = clean_data(ref_data, params)

In [22]:
df_clean.head(10)

,booking_id,number_of_adults,number_of_children,number_of_weekend_nights,number_of_week_nights,type_of_meal,room_type,lead_time,market_segment_type,average_price,special_requests,date_of_reservation,booking_status
0,INN00001,1,1,2,5,Meal Plan 1,Room_Type 1,224,Offline,88.00,0,2015-10-02,Not_Canceled
1,INN00002,1,0,1,3,Not Selected,Room_Type 1,5,Online,106.68,1,2018-11-06,Not_Canceled
2,INN00003,2,1,1,3,Meal Plan 1,Room_Type 1,1,Online,50.00,0,2018-02-28,Canceled
3,INN00004,1,0,0,2,Meal Plan 1,Room_Type 1,211,Online,100.00,1,2017-05-20,Canceled
4,INN00005,1,0,1,2,Not Selected,Room_Type 1,48,Online,77.00,0,2018-04-11,Canceled
6,INN00007,1,1,1,4,Meal Plan 1,Room_Type 1,34,Online,107.55,1,2017-10-15,Not_Canceled
7,INN00008,3,0,1,3,Meal Plan 1,Room_Type 4,83,Online,105.61,1,2018-12-26,Not_Canceled
8,INN00009,1,1,0,4,Meal Plan 1,Room_Type 1,121,Offline,96.90,1,2018-07-06,Not_Canceled
9,INN00010,2,0,0,5,Meal Plan 1,Room_Type 4,44,Online,133.44,3,2018-10-18,Not_Canceled
10,INN00011,1,0,1,0,Not Selected,Room_Type 1,0,Online,85.03,0,2018-09-11,Not_Canceled


## Test Preprocessing Train

In [55]:
from pathlib import Path
import pytest
import pandas as pd
import numpy as np



def test_clean_data_type():
    df = pd.read_csv('../tests/pipelines/sample/sample.csv') 
    params = {'preprocessing': {'drop_columns': ["car_parking_space", "repeated"]}}
    df_transformed, describe_to_dict_verified  = clean_data(df, params)
    isinstance(describe_to_dict_verified, dict)

# checking for null values after cleaning
def test_clean_data_null(): 
    # create test data
    data = {
        'booking_id': ['N1', 'N2', 'N3', 'N4', 'N5'],
        'number_of_adults': [2, 1, 2, np.nan, 2],
        'number_of_children': [0, 1, np.nan, 0, 2],
        'number_of_weekend_nights': [1, 2, 1, 0, np.nan],
        'number_of_week_nights': [2, 3, 1, 0, 2],
        'type_of_meal': ['Meal Plan 1', 'Meal Plan 2', np.nan, 'Meal Plan 1', 'Meal Plan 1'],
        'car_parking_space': [0, 1, 0, 1, 1],
        'room_type': ['Room_Type 1', 'Room_Type 4', 'Room_Type 1', np.nan, 'Room_Type 2'],
        'lead_time': [30, 90, 45, 10, np.nan],
        'market_segment_type': ['Online', 'Offline', 'Online', 'Corporate', np.nan],
        'repeated': [0, 1, 0, 0, 1],
        'p_c': [0.2, 0.4, 0.3, 0.1, 0.5],
        'p_not_c': [0.8, 0.6, 0.7, 0.9, 0.5],
        'average_price': [100.0, 120.5, np.nan, 90.0, 105.0],
        'special_requests': [1, 0, 2, 0, 1],
        'date_of_reservation': [
            pd.Timestamp('2018-07-10'),
            pd.NaT,  # missing date
            pd.Timestamp('2018-09-15'),
            pd.Timestamp('2018-08-20'),
            pd.Timestamp('2018-09-01')
        ],
        'booking_status': ['Not_Canceled', 'Canceled', 'Canceled', 'Not_Canceled', 'Not_Canceled']
    }
    df = pd.DataFrame(data)

    params = {'preprocessing': {'drop_columns': ["car_parking_space", "repeated"]}}

    df_transformed, describe_to_dict_verified  = clean_data(df, params)
    assert [col for col in df_transformed.columns if df_transformed[col].isnull().any()] == [], "Missing values not dropped"

# checking for invalid bookings after cleaning
def test_clean_data_valid_bookings():
    #create test data
    data = {
        'booking_id': ['C1', 'C2', 'C3', 'C4', 'C5'],
        'number_of_adults': [0, 2, 1, 2, 2],                     # C1: both zero, C3: one zero, C5: both non-zero
        'number_of_children': [0, 1, 0, 1, 2],
        'number_of_weekend_nights': [1, 0, 1, 0, 2],             # C2: one zero, C4: both zero, C5: both non-zero
        'number_of_week_nights': [2, 3, 2, 0, 4],
        'type_of_meal': ['Meal Plan 1', 'Meal Plan 2', 'Meal Plan 1', 'Meal Plan 3', 'Meal Plan 2'],
        'car_parking_space': [1, 0, 1, 0, 1],
        'room_type': ['Room_Type 1', 'Room_Type 4', 'Room_Type 1', 'Room_Type 3', 'Room_Type 2'],
        'lead_time': [20, 50, 15, 5, 30],
        'market_segment_type': ['Online', 'Corporate', 'Offline', 'Online', 'Offline'],
        'repeated': [0, 1, 0, 0, 1],
        'p_c': [0.1, 0.3, 0.25, 0.05, 0.4],
        'p_not_c': [0.9, 0.7, 0.75, 0.95, 0.6],
        'average_price': [110.0, 130.0, 95.0, 80.0, 125.0],
        'special_requests': [1, 0, 2, 0, 1],
        'date_of_reservation': pd.to_datetime(['2018-08-01', '2018-08-15', '2018-09-01', '2018-09-10', '2018-09-18']),
        'booking_status': ['Not_Canceled', 'Canceled', 'Canceled', 'Not_Canceled', 'Canceled']
    }
    df = pd.DataFrame(data)
    params = {'preprocessing': {'drop_columns': ["car_parking_space", "repeated"]}}

    df_transformed, describe_to_dict_verified  = clean_data(df, params)
    assert not ((df_transformed["number_of_week_nights"] == 0) & (df_transformed["number_of_weekend_nights"] == 0)).any(), "Invalid total of nights"
    assert not ((df_transformed["number_of_children"] == 0) & (df_transformed["number_of_adults"] == 0)).any(), "Invalid total of guests"


# checking for dropping requested columns:
def test_clean_data_drop_columns():
    df = pd.read_csv('../tests/pipelines/sample/sample.csv') 
    params = {'preprocessing': {'drop_columns': ["car_parking_space", "repeated"]}}
    df_transformed, describe_to_dict_verified  = clean_data(df, params)
    assert df_transformed.shape[1] == df.shape[1] - len(params['preprocessing']['drop_columns']), "drop_columns not dropped"

In [56]:
test_clean_data_type()

In [57]:
test_clean_data_null()

In [59]:
test_clean_data_valid_bookings()

In [60]:
test_clean_data_drop_columns()

## Feature Engineering

We want to create a new feature: season. Based on the date of reservation and lead time, we can establish the first day of stay. Assume stay in June, July, August and December - high season, other months - low season

In [25]:
def add_season(data: pd.DataFrame):
    
    df = data.copy()
 
    #new feature: season
    df['season'] = pd.to_datetime(df['date_of_reservation']) + pd.to_timedelta(df['lead_time'], unit='d')
    df['season'] = df['season'].dt.month.apply(lambda m: 'high' if m in [6,7,8,12] else 'low')

    return df

In [26]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

def feature_engineer( data: pd.DataFrame):

    #add season feature
    df = add_season(data)

    #map target variable
    if "booking_status" in df.columns:
        df["booking_status"] = df["booking_status"].map({"Canceled": 1, "Not_Canceled": 0})

    #replace underrepresented categories by Other:
    df['room_type'] = df['room_type'].replace({
    'Room_Type 2': 'Other',
    'Room_Type 3': 'Other',
    'Room_Type 5': 'Other',
    'Room_Type 6': 'Other',
    'Room_Type 7': 'Other'
    })

    df['market_segment_type'] = df['market_segment_type'].replace({
    'Aviation': 'Other',
    'Complementary': 'Other'
    })

    # separate target variable and columns that don't need encoding/scaling
    exclude_cols = ['booking_id', 'date_of_reservation', 'booking_status']


    numerical_features = df.select_dtypes(exclude=['object', 'string', 'category']).columns.difference(exclude_cols).tolist()
    categorical_features = df.select_dtypes(include=['object', 'string', 'category']).columns.difference(exclude_cols).tolist()

    # use one hot encoder for categorical geatures
    OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    OH_cols= pd.DataFrame(OH_encoder.fit_transform(df[categorical_features]))

    OH_cols.columns = OH_encoder.get_feature_names_out(categorical_features)

    # put back the index
    OH_cols.index = df.index

    # Remove categorical columns (will replace with one-hot encoding)
    rest_df = df.drop(categorical_features, axis=1)

    # Add one-hot encoded columns to numerical features
    df_final_tree = pd.concat([rest_df, OH_cols], axis=1)

    # scale numerical features
    ST_scaler = StandardScaler()


    scaled_num_array = ST_scaler.fit_transform(df[numerical_features])
    scaled_num_df = pd.DataFrame(scaled_num_array, columns=numerical_features, index=df.index)

    # create a dataframe with one-hot-encoded categorical and scaled numerical features
    df_final_lr = df_final_tree.copy()
    df_final_lr[numerical_features] = scaled_num_df[numerical_features]

    # set booking_id as index
    df_final_tree.set_index("booking_id", inplace=True)
    df_final_lr.set_index("booking_id", inplace=True)

    

    return df_final_tree, df_final_lr, OH_encoder, ST_scaler

In [27]:
df_tr, df_lr, encod, scal = feature_engineer(df_clean)

In [28]:
df_tr.head(10)

,number_of_adults,number_of_children,number_of_weekend_nights,number_of_week_nights,lead_time,average_price,special_requests,date_of_reservation,booking_status,market_segment_type_Corporate,...,market_segment_type_Online,market_segment_type_Other,room_type_Other,room_type_Room_Type 1,room_type_Room_Type 4,season_high,season_low,type_of_meal_Meal Plan 1,type_of_meal_Meal Plan 2,type_of_meal_Not Selected
booking_id,,,,,,,,,,,,,,,,,,,,,
INN00001,1,1,2,5,224,88.00,0,2015-10-02,0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
INN00002,1,0,1,3,5,106.68,1,2018-11-06,0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
INN00003,2,1,1,3,1,50.00,0,2018-02-28,1,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
INN00004,1,0,0,2,211,100.00,1,2017-05-20,1,0.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
INN00005,1,0,1,2,48,77.00,0,2018-04-11,1,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
INN00007,1,1,1,4,34,107.55,1,2017-10-15,0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
INN00008,3,0,1,3,83,105.61,1,2018-12-26,0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
INN00009,1,1,0,4,121,96.90,1,2018-07-06,0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
INN00010,2,0,0,5,44,133.44,3,2018-10-18,0,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0


In [29]:
df_tr.columns

Index(['number_of_adults', 'number_of_children', 'number_of_weekend_nights',
       'number_of_week_nights', 'lead_time', 'average_price',
       'special_requests', 'date_of_reservation', 'booking_status',
       'market_segment_type_Corporate', 'market_segment_type_Offline',
       'market_segment_type_Online', 'market_segment_type_Other',
       'room_type_Other', 'room_type_Room_Type 1', 'room_type_Room_Type 4',
       'season_high', 'season_low', 'type_of_meal_Meal Plan 1',
       'type_of_meal_Meal Plan 2', 'type_of_meal_Not Selected'],
      dtype='object')

In [30]:
df_tr.isnull().sum()

number_of_adults                 0
number_of_children               0
number_of_weekend_nights         0
number_of_week_nights            0
lead_time                        0
average_price                    0
special_requests                 0
date_of_reservation              0
booking_status                   0
market_segment_type_Corporate    0
market_segment_type_Offline      0
market_segment_type_Online       0
market_segment_type_Other        0
room_type_Other                  0
room_type_Room_Type 1            0
room_type_Room_Type 4            0
season_high                      0
season_low                       0
type_of_meal_Meal Plan 1         0
type_of_meal_Meal Plan 2         0
type_of_meal_Not Selected        0
dtype: int64

In [31]:
df_lr.head(10)

,number_of_adults,number_of_children,number_of_weekend_nights,number_of_week_nights,lead_time,average_price,special_requests,date_of_reservation,booking_status,market_segment_type_Corporate,...,market_segment_type_Online,market_segment_type_Other,room_type_Other,room_type_Room_Type 1,room_type_Room_Type 4,season_high,season_low,type_of_meal_Meal Plan 1,type_of_meal_Meal Plan 2,type_of_meal_Not Selected
booking_id,,,,,,,,,,,,,,,,,,,,,
INN00001,-1.622889,2.669144,1.323168,1.947978,2.072440,-0.496692,-0.789910,2015-10-02,0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
INN00002,-1.622889,-0.240221,0.186423,0.546523,-1.006239,0.185818,0.498460,2018-11-06,0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
INN00003,0.304398,2.669144,0.186423,0.546523,-1.062471,-1.885097,-0.789910,2018-02-28,1,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
INN00004,-1.622889,-0.240221,-0.950323,-0.154204,1.889687,-0.058249,0.498460,2017-05-20,1,0.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
INN00005,-1.622889,-0.240221,0.186423,-0.154204,-0.401750,-0.898599,-0.789910,2018-04-11,1,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
INN00007,-1.622889,2.669144,0.186423,1.247251,-0.598560,0.217605,0.498460,2017-10-15,0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
INN00008,2.231684,-0.240221,0.186423,0.546523,0.090276,0.146724,0.498460,2018-12-26,0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
INN00009,-1.622889,2.669144,-0.950323,1.247251,0.624476,-0.171513,0.498460,2018-07-06,0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
INN00010,0.304398,-0.240221,-0.950323,1.947978,-0.457981,1.163547,3.075199,2018-10-18,0,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0


In [32]:
df_lr.isnull().sum()

number_of_adults                 0
number_of_children               0
number_of_weekend_nights         0
number_of_week_nights            0
lead_time                        0
average_price                    0
special_requests                 0
date_of_reservation              0
booking_status                   0
market_segment_type_Corporate    0
market_segment_type_Offline      0
market_segment_type_Online       0
market_segment_type_Other        0
room_type_Other                  0
room_type_Room_Type 1            0
room_type_Room_Type 4            0
season_high                      0
season_low                       0
type_of_meal_Meal Plan 1         0
type_of_meal_Meal Plan 2         0
type_of_meal_Not Selected        0
dtype: int64

In [61]:
df_lr.shape

(30714, 21)

In [62]:
df_tr.shape

(30714, 21)

## Data Unit Tests After Preprocessing

In [52]:
test_report = pd.read_csv('../data/08_reporting/data_tests.csv')

In [53]:
display(test_report)

,Success,Expectation Type,Column,Column Pair,Max Value,Min Value,Element Count,Unexpected Count,Unexpected Percent,Value Set,Unexpected Value,Observed Value
0,True,expect_column_values_to_be_unique,booking_id,"('', '')",NaN,NaN,36248.0,0.0,0.000000,NaN,[],NaN
1,True,expect_column_to_exist,booking_status,"('', '')",NaN,NaN,NaN,NaN,NaN,NaN,[],NaN
2,True,expect_column_distinct_values_to_be_in_set,type_of_meal,"('', '')",NaN,NaN,NaN,NaN,NaN,"['Meal Plan 1', 'Meal Plan 2', 'Meal Plan 3', ...",[],"['Meal Plan 1', 'Meal Plan 2', 'Meal Plan 3', ..."
3,True,expect_column_distinct_values_to_be_in_set,room_type,"('', '')",NaN,NaN,NaN,NaN,NaN,"['Room_Type 1', 'Room_Type 2', 'Room_Type 3', ...",[],"['Room_Type 1', 'Room_Type 2', 'Room_Type 3', ..."
4,True,expect_column_distinct_values_to_be_in_set,market_segment_type,"('', '')",NaN,NaN,NaN,NaN,NaN,"['Online', 'Offline', 'Corporate', 'Complement...",[],"['Aviation', 'Complementary', 'Corporate', 'Of..."
5,True,expect_column_values_to_be_between,number_of_adults,"('', '')",4.0,0.0,36248.0,0.0,0.000000,NaN,[],NaN
6,True,expect_column_values_to_be_between,number_of_children,"('', '')",10.0,0.0,36248.0,0.0,0.000000,NaN,[],NaN
7,True,expect_column_values_to_be_between,number_of_week_nights,"('', '')",17.0,0.0,36248.0,0.0,0.000000,NaN,[],NaN
8,True,expect_column_values_to_be_between,number_of_weekend_nights,"('', '')",7.0,0.0,36248.0,0.0,0.000000,NaN,[],NaN
9,False,expect_column_values_to_be_between,lead_time,"('', '')",365.0,0.0,36248.0,243.0,0.670382,NaN,[],NaN


In [54]:
test_report[test_report['Success'] == False]

,Success,Expectation Type,Column,Column Pair,Max Value,Min Value,Element Count,Unexpected Count,Unexpected Percent,Value Set,Unexpected Value,Observed Value
9,False,expect_column_values_to_be_between,lead_time,"('', '')",365.0,0.0,36248.0,243.0,0.670382,NaN,[],NaN
10,False,expect_column_values_to_be_between,average_price,"('', '')",500.0,0.0,36248.0,1.0,0.002759,NaN,[],NaN
